<a href="https://colab.research.google.com/github/econ105/AI/blob/main/silvergoldinvestment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import yfinance as yf
import pandas as pd

# ----------------------------------
# 1. Define your investment universe
# ----------------------------------
# You can change / add tickers here:
# SLV – Silver, GLD – Gold, SPY – S&P 500, TLT – US long-term Treasuries, GDX – Gold miners
tickers = ["SLV", "GLD", "SPY", "TLT", "GDX"]

# ----------------------------------
# 2. Download price history
# ----------------------------------
prices = yf.download(
    tickers=tickers,
    period="5y",
    interval="1d",
    auto_adjust=True,
    progress=False
)

# Keep only the Close column and compute daily returns
close_px = prices["Close"]
daily_returns = close_px.pct_change().dropna()

# ----------------------------------
# 3. Pull fundamentals for each ticker
# ----------------------------------
# Fields you care about (add or remove as needed)
fund_fields = [
    "longName",
    "symbol",
    "category",         # for ETFs
    "fundFamily",
    "marketCap",
    "trailingPE",
    "forwardPE",
    "priceToBook",
    "beta",
    "dividendYield",
    "yield",            # fund distribution yield
    "expenseRatio",     # some ETFs expose this
]

fundamentals = []

for tk in tickers:
    t = yf.Ticker(tk)
    info = t.info  # dictionary of fundamentals/metadata

    row = {f: info.get(f) for f in fund_fields}
    fundamentals.append(row)

fundamentals_df = pd.DataFrame(fundamentals)

print("Fundamentals table:")
print(fundamentals_df)

# ----------------------------------
# 4. Simple performance stats
# ----------------------------------
perf_stats = []

for tk in tickers:
    ret = daily_returns[tk]
    ann_return = (1 + ret.mean())**252 - 1
    ann_vol = ret.std() * (252**0.5)

    perf_stats.append({
        "symbol": tk,
        "annual_return": ann_return,
        "annual_volatility": ann_vol,
        "sharpe_ratio_(no_rf)": ann_return / ann_vol if ann_vol != 0 else None
    })

perf_df = pd.DataFrame(perf_stats)
perf_df = perf_df.merge(
    fundamentals_df[["symbol", "longName"]],
    on="symbol",
    how="left"
)

print("\nPerformance + key info:")
print(perf_df)

# ----------------------------------
# 5. Example: focus on gold & silver
# ----------------------------------
subset = perf_df[perf_df["symbol"].isin(["GLD", "SLV"])]
print("\nGold vs Silver snapshot:")
print(subset)


Fundamentals table:
                              longName symbol                category  \
0                 iShares Silver Trust    SLV     Commodities Focused   
1                     SPDR Gold Shares    GLD     Commodities Focused   
2  State Street SPDR S&P 500 ETF Trust    SPY             Large Blend   
3   iShares 20+ Year Treasury Bond ETF    TLT         Long Government   
4               VanEck Gold Miners ETF    GDX  Equity Precious Metals   

                           fundFamily     marketCap  trailingPE  forwardPE  \
0                             iShares  2.575899e+10         NaN        NaN   
1  State Street Investment Management  1.158205e+11         NaN        NaN   
2  State Street Investment Management  6.350776e+11   28.015951        NaN   
3                             iShares  9.558160e+09         NaN    -4356.5   
4                              VanEck           NaN   26.560728        NaN   

   priceToBook  beta  dividendYield   yield expenseRatio  
0     3.53382